# Extended Applications

EvoX facilitates efficient exploration of complex optimization landscapes, effective tackling of black-box optimization challenges, and deep dives into neuroevolution with Brax. Thus, it is talented in extended applications. 
Here we will show an example of Neuroevolution Tasks using EvoX and Brax.

In [13]:
# install EvoX, skip it if you have already installed EvoX
from importlib.util import find_spec

if find_spec("evox") is None:
    %pip install evox

In [14]:
# The dependent packages or functions in this example
import time

import torch
import torch.nn as nn

from evox.algorithms import PSO
from evox.problems.neuroevolution.brax import BraxProblem
from evox.utils import ParamsAndVector
from evox.workflows import EvalMonitor, StdWorkflow

## Use EvoX to solve Neuroevolution Tasks
Neuroevolution is an optimization method that combines neural networks with evolutionary algorithms to evolve the structure and parameters of neural networks. By simulating natural selection and genetic mechanisms, Neuroevolution aims to optimize neural network architectures and weights, addressing complex problems such as game AI, robotic control, and more.

In our example of neuroevolution tasks, Brax is needed. So it is recommended to install Brax if you want to reproduce this example.

### What is Brax

Brax is a fast and fully differentiable physics engine used for research and development of robotics, human perception, materials science, reinforcement learning, and other simulation-heavy applications. 

For more information, you can browse the [Github of Brax](https://github.com/google/brax).

We will demonstrate a "hopper" environment of Brax.

### Design a neural network class

To start with, we need to decide which neural network we are about to construct.

Here we will give a simple Convolutional Neural Network (CNN) class. 

In [15]:
# Construct an CNN using Torch.
# This CNN has 2 layers.


class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(nn.Linear(11, 4), nn.Tanh(), nn.Linear(4, 3))

    def forward(self, x):
        x = self.features(x)
        return x

### Initiate a model

Through the ``SimpleCNN`` class, we can initiate a CNN model.

In [16]:
# Make sure that the model is on the same device, better to be on the GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
# Reset the random seed
seed = 1234
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Initialize the CNN model
model = SimpleCNN().to(device)

We can compute the total number of the model parameters, and check if the model id initialized correctly. If everything goes well, we will see the total number is 63.

In [17]:
# Test if the model is initialized correctly by computing the number of model parameters
for p in model.parameters():
    p.requires_grad = False
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of model parameters: {total_params}")

Total number of model parameters: 63


We can also test the dimoension of the inputs and outputs. If everything goes well, 11 inputs will obtain 3 outputs.

In [18]:
inputs = torch.rand(1, 11, device=device)
outputs = model(inputs)
print("Test model output:", outputs)

Test model output: tensor([[-0.0559,  0.4188,  0.2301]])


if we check the weights for this network, we will see that it's group of parameter sets, which EC algorithms cannot directly work with data in this format:

In [19]:
model.state_dict()

OrderedDict([('features.0.weight',
              tensor([[-0.2840, -0.0592, -0.1448, -0.0804, -0.2664,  0.1210, -0.2703, -0.0192,
                        0.1048, -0.1016,  0.1711],
                      [ 0.0380,  0.1657,  0.1935, -0.1331,  0.1096, -0.1304,  0.0945, -0.1575,
                        0.1395,  0.0610, -0.1180],
                      [-0.1479,  0.0780,  0.2813,  0.1447, -0.0291, -0.0146,  0.1714, -0.2096,
                        0.1002, -0.0999,  0.1744],
                      [-0.1076,  0.0149,  0.1018,  0.2072, -0.0443,  0.2751, -0.2551, -0.0538,
                       -0.3007,  0.0250,  0.0856]])),
             ('features.0.bias', tensor([-0.1221,  0.1252, -0.0489, -0.2620])),
             ('features.2.weight',
              tensor([[ 0.3839,  0.3083,  0.2528,  0.3988],
                      [ 0.1839,  0.2658,  0.4149, -0.1007],
                      [-0.3900, -0.2459, -0.0667, -0.0549]])),
             ('features.2.bias', tensor([-0.0034,  0.2865,  0.1604]))])

Fortunately, EvoX provides some useful utilities to help us bridge the gap, and in this case, we have `ParamsAndVector` class to help us convert a tree-like struct into a vector and back.

### Initiate an adapter

An adapter can help us convert the data back-and-forth.

- `to_vector` can convert a parameters dictionary to a vector.
- `to_params` can convert a vector back to a parameters dictionary.

There are also batched version conversion.

- `batched_to_vector` can convert a batched parameters dictionary to a batch of vectors.
- `batched_to_params` can convert a batch of vectors back to a batched parameters dictionary.

In [20]:
adapter = ParamsAndVector(dummy_model=model)
feature_weights = adapter.to_vector(model.state_dict())
feature_weights

tensor([-0.2840, -0.0592, -0.1448, -0.0804, -0.2664,  0.1210, -0.2703, -0.0192,
         0.1048, -0.1016,  0.1711,  0.0380,  0.1657,  0.1935, -0.1331,  0.1096,
        -0.1304,  0.0945, -0.1575,  0.1395,  0.0610, -0.1180, -0.1479,  0.0780,
         0.2813,  0.1447, -0.0291, -0.0146,  0.1714, -0.2096,  0.1002, -0.0999,
         0.1744, -0.1076,  0.0149,  0.1018,  0.2072, -0.0443,  0.2751, -0.2551,
        -0.0538, -0.3007,  0.0250,  0.0856, -0.1221,  0.1252, -0.0489, -0.2620,
         0.3839,  0.3083,  0.2528,  0.3988,  0.1839,  0.2658,  0.4149, -0.1007,
        -0.3900, -0.2459, -0.0667, -0.0549, -0.0034,  0.2865,  0.1604])

With an adapter, we can set out to do this Neuroevolution Task like what we did in the Quick Start.

## Set up the running process

### Initiate an algorithm and a problem

We still initiate a PSO algorithm, and the problem is a Brax problem in "hopper" environment.

In [21]:
# Set the population size
POP_SIZE = 10

# Get the bound of the PSO algorithm
model_params = dict(model.named_parameters())
pop_center = adapter.to_vector(model_params)
lower_bound = pop_center - 1
upper_bound = pop_center + 1

# Initialize the PSO, and you can also use any other algorithms
algorithm = PSO(
    pop_size=POP_SIZE,
    lb=lower_bound,
    ub=upper_bound,
    device=device,
)
algorithm.setup()

# Initialize the Brax problem
problem = BraxProblem(
    policy=model,
    env_name="hopper",
    max_episode_length=1000,
    num_episodes=3,
    pop_size=POP_SIZE,
    device=device,
)

Notice:
- `max_episode_length` is the maximum number of steps for each episode.

- `num_episodes` is the number of episodes to run for each evaluation.

In this case, we will be using 1000 steps for each episode, and the average reward of 3 episodes will be returned as the fitness value.

### Set an monitor

In [22]:
# set an monitor, and it can record the top 3 best fitnesses
pop_monitor = EvalMonitor(
    topk=3,
    device=device,
)
pop_monitor.setup()

EvalMonitor()

### Initiate an workflow

In [23]:
# Initiate an workflow
workflow = StdWorkflow(opt_direction="max")
workflow.setup(
    algorithm=algorithm,
    problem=problem,
    solution_transform=adapter,
    monitor=pop_monitor,
    device=device,
)

### Run the workflow

Run the workflow and see the magic!

```{note}
The following block will take around 1 minute to run.
The time may vary depending on your hardware.
```

In [24]:
# Set the maximum number of generations
max_generation = 3

# Run the workflow
for index in range(max_generation):
    print(f"In generation {index}:")
    t = time.time()
    workflow.step()
    print(f"\tTime elapsed: {time.time() - t: .4f}(s).")
    monitor: EvalMonitor = workflow.get_submodule("monitor")
    print(f"\tTop fitness: {monitor.topk_fitness}")
    best_params = adapter.to_params(monitor.topk_solutions[0])
    print(f"\tBest params: {best_params}")

In generation 0:
	Time elapsed:  18.4627(s).
	Top fitness: tensor([-684.6812, -603.0337, -597.3137])
	Best params: {'features.0.weight': tensor([[-0.4586,  0.1369, -0.6505, -0.1045, -0.6985, -0.2686, -0.4071, -0.4004,
          0.8730,  0.1611,  1.0660],
        [ 0.9409,  0.3292, -0.2226,  0.4750,  0.0748,  0.6098, -0.1190,  0.7268,
          0.2716, -0.1611, -0.7704],
        [ 0.5922,  0.2081,  0.6272,  0.5489, -0.0808, -0.5174, -0.1271, -0.2541,
          0.7212, -0.1730, -0.0791],
        [-0.2435,  0.1791,  0.9000,  0.9618, -0.1028,  0.4076,  0.6518,  0.3856,
         -0.4062,  0.6178,  0.5410]]), 'features.0.bias': tensor([-0.3906,  0.5298,  0.0304, -0.6333]), 'features.2.weight': tensor([[ 0.3054,  1.2495,  0.5532, -0.4268],
        [ 0.8211,  0.4807,  0.9068, -0.0965],
        [ 0.0078,  0.0530,  0.8113,  0.4728]]), 'features.2.bias': tensor([-0.4410,  0.8106,  0.1011])}
In generation 1:
	Time elapsed:  5.4289(s).
	Top fitness: tensor([-684.6812, -658.5977, -611.5485])
	Best p

```{note}
The PSO wasn’t specialized for this type of tasks, so its performance limitations here are expected. Here we just show an example, and hope you can use a quantity of more effective algorithms in EvoX and enjoy your time!
```